In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# --- (10일차 추가) StandardScaler 임포트 ---
from sklearn.preprocessing import StandardScaler

# --- 1. (8일차) Custom Dataset 클래스 (10일차 수정) ---
class JeonseDataset(Dataset):

    # __init__이 csv_path 외에 'scaler' 객체와 'is_train' 플래그를 받도록 수정
    def __init__(self, csv_path, scaler=None, is_train=True):
        df = pd.read_csv(csv_path)

        # 'risk_label'을 제외한 모든 열을 features로 간주
        feature_cols = df.columns.drop('risk_label')

        # --- (10일차 핵심) 전처리 로직 ---
        if is_train:
            # 훈련용 데이터라면, 새로운 Scaler를 생성하고 fit/transform
            self.scaler = StandardScaler()
            self.features = self.scaler.fit_transform(df[feature_cols])
        else:
            # 테스트용 데이터라면, 훈련 때 사용한 scaler를 그대로 받아 transform
            if scaler is None:
                raise ValueError("테스트 데이터셋은 반드시 훈련용 scaler를 받아야 합니다.")
            self.scaler = scaler
            self.features = self.scaler.transform(df[feature_cols])
        # ------------------------------------

        self.labels = df['risk_label'].values
        print(f"{'훈련' if is_train else '테스트'} 데이터셋 로드 완료. Scaler 적용됨.")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # self.features는 이미 NumPy 배열이자 스케일링이 완료된 상태
        feature = self.features[idx]
        label = self.labels[idx]

        feature_tensor = torch.tensor(feature, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        return feature_tensor, label_tensor.view(1)

    # (10일차 추가) 나중에 테스트할 때 사용할 scaler를 반환하는 헬퍼 함수
    def get_scaler(self):
        return self.scaler

# --- 2. (5일차) 모델(MLP) 클래스 (9일차와 동일) ---
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.layer1(x); out = self.relu(out)
        out = self.layer2(out); out = self.sigmoid(out)
        return out

# --- 3. (10일차 수정) 학습 준비 ---

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# (Hyperparameters - 9일차와 동일)
input_dim = 3; hidden_dim = 8; output_dim = 1
learning_rate = 0.001; batch_size = 16; num_epochs = 50

# (데이터 준비 - 10일차 수정)
csv_file_path = 'dummy_data.csv'
# is_train=True로 설정하여 Dataset 내부에서 Scaler를 fit 하도록 함
train_dataset = JeonseDataset(csv_file_path, is_train=True)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# (모델, 손실함수, 옵티마이저 - 9일차와 동일)
model = SimpleMLP(input_dim, hidden_dim, output_dim).to(device)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("--- 스케일링 적용 후 학습 시작 ---")

# --- 4. (9일차와 동일) 중첩 학습 루프 ---
for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    for features_batch, labels_batch in train_data_loader:
        features_batch = features_batch.to(device)
        labels_batch = labels_batch.to(device)

        prediction = model(features_batch)
        loss = loss_fn(prediction, labels_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        predicted_labels = prediction.round()
        correct = (predicted_labels == labels_batch).sum().item()
        epoch_accuracy += correct

    avg_loss = epoch_loss / len(train_data_loader)
    avg_accuracy = epoch_accuracy / len(train_dataset)

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1:3d}/{num_epochs}] | Loss: {avg_loss:.4f} | Accuracy: {avg_accuracy * 100:.2f}%")

print("--- 학습 완료 ---")

# --- 5. (10일차) 스케일링 적용 후 예측 테스트 ---
# 훈련에 사용된 scaler를 가져옴
fitted_scaler = train_dataset.get_scaler()

# 9일차와 동일한 테스트 샘플
test_data_raw = np.array([[0.9, 0.5, 10.0]]) # [전세가율, 근저당, 10년차]

# (핵심!) 테스트 샘플에도 '반드시' scaler.transform()을 적용
test_data_scaled = fitted_scaler.transform(test_data_raw)

print(f"\n원본 테스트 데이터: {test_data_raw}")
print(f"스케일링된 테스트 데이터: {test_data_scaled}")

# 스케일링된 데이터를 텐서로 변환하여 모델에 입력
test_tensor = torch.tensor(test_data_scaled, dtype=torch.float32).to(device)
risk_prob = model(test_tensor)
print(f"테스트 샘플의 위험 확률: {risk_prob.item() * 100:.2f}%")

훈련 데이터셋 로드 완료. Scaler 적용됨.
--- 스케일링 적용 후 학습 시작 ---
Epoch [  5/50] | Loss: 0.6867 | Accuracy: 58.00%
Epoch [ 10/50] | Loss: 0.6789 | Accuracy: 57.00%
Epoch [ 15/50] | Loss: 0.6834 | Accuracy: 57.00%
Epoch [ 20/50] | Loss: 0.6754 | Accuracy: 57.00%
Epoch [ 25/50] | Loss: 0.6749 | Accuracy: 57.00%
Epoch [ 30/50] | Loss: 0.6855 | Accuracy: 57.00%
Epoch [ 35/50] | Loss: 0.6680 | Accuracy: 57.00%
Epoch [ 40/50] | Loss: 0.6641 | Accuracy: 57.00%
Epoch [ 45/50] | Loss: 0.6627 | Accuracy: 55.00%
Epoch [ 50/50] | Loss: 0.6711 | Accuracy: 56.00%
--- 학습 완료 ---

원본 테스트 데이터: [[ 0.9  0.5 10. ]]
스케일링된 테스트 데이터: [[ 0.35400675  0.46726213 -0.61904123]]
테스트 샘플의 위험 확률: 58.76%


C:\Users\dnwls\PycharmProjects\JupyterProject\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
